In [6]:
from datetime import datetime, time, timedelta, timezone
from typing import Any, Dict

from pydantic import AwareDatetime, BaseModel, WrapSerializer
from typing_extensions import Annotated


class EventDatetime(BaseModel):
    start: datetime
    end: datetime
    aware_datetime: AwareDatetime = datetime(1, 2, 3)
    deltat: timedelta = timedelta(1, 2)
    time_of_date: time = time(12, 45)
    inty: int = 1


def convert_to_utc(value: Any, handler, info) -> Dict[str, Any]:
    # Note that `helper` can actually help serialize the `value` for further custom serialization in case it's a subclass.
    partial_result = handler(value, info)
    fn = (
        lambda v: "datetime.fromisoformat(v).astimezone(timezone.utc)"
        if isinstance(v, datetime)
        else v
    )
    if info.mode == "json":
        return {k: fn(v) for k, v in partial_result.items()}
    fn = lambda v: v.astimezone(timezone.utc) if isinstance(v, datetime) else v
    return {k: fn(v) for k, v in partial_result.items()}


UTCEventDatetime = Annotated[EventDatetime, WrapSerializer(convert_to_utc)]


class EventModel(BaseModel):
    event_datetime: UTCEventDatetime


dt = EventDatetime(start="2024-01-01T07:00:00-08:00", end="2024-01-03T20:00:00+06:00")
event = EventModel(event_datetime=dt)
print(event.model_dump())
"""
{
    'event_datetime': {
        'start': datetime.datetime(
            2024, 1, 1, 15, 0, tzinfo=datetime.timezone.utc
        ),
        'end': datetime.datetime(
            2024, 1, 3, 14, 0, tzinfo=datetime.timezone.utc
        ),
    }
}
"""

print(event.model_dump_json())

{'event_datetime': {'start': datetime.datetime(2024, 1, 1, 15, 0, tzinfo=datetime.timezone.utc), 'end': datetime.datetime(2024, 1, 3, 14, 0, tzinfo=datetime.timezone.utc), 'aware_datetime': datetime.datetime(1, 2, 3, 0, 1, 15, tzinfo=datetime.timezone.utc), 'deltat': datetime.timedelta(days=1, seconds=2), 'time_of_date': datetime.time(12, 45), 'inty': 1}}
{"event_datetime":{"start":"2024-01-01T07:00:00-08:00","end":"2024-01-03T20:00:00+06:00","aware_datetime":"0001-02-03T00:00:00","deltat":"P1DT2S","time_of_date":"12:45:00","inty":1}}
